# Ranking Systems

Every year, Division 1 college football is consumed with debate over the team rankings.  Starting in 1998, these rankings directly impacted which teams would face off in the BCS Championship game.  Starting in 2014, the rankings impacted the selection of the top 4 teams for the BCS playoff.  College football is a perfect example of the need for a ranking system but the approaches laid out in this notebook are potentially universal: they can be applied to any sport.  

It is important to clarify two terms that will be used throughout: _rating_ and _ranking_.  A _rating_ is a numeric value that quantifies the performance of a player or team.  A _ranking_ is an ordering of the players/teams according to a rating metric.

In [ ]:
%run ../../utils/notebook_setup.py

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Incomplete Tournaments

In the mathematics of graphs (not plots but networks of nodes and edges), a _Complete Tournament_ is what is commonly known as _Round Robin_: every team plays every other team.  In the NBA and the NHL, each pair of teams play at least twice.  In MLB, a team is only guaranteed to play teams within its own league.  And within the NFL, a team is only guaranteed to play teams within its division.

Division 1A college football has 129 teams.  They play 12 games each.  This is about as far from a Round Robin as it gets.  Given the diverse performance and relative lack of games played, how do we build a ranking under these conditions?

This notebook will cover three approaches:
+ Elo
+ Matrix/Regression
+ Graph/Network

## 1. Elo Rating

Suppose we have two teams, $A$ and $B$, with ratings $R_A$ and $R_B$, respectively.

### Expected Outcome
With a win as 1 and a loss as 0, we can compute the expected outcome for $A$ (also a probability of winning),
$$
    E_A = \frac 1 {1 + 10^{(R_B - R_A)/400}} = \frac{Q_A}{Q_A + Q_B}
$$
and the expected outcome for $B$,
$$
    E_B = \frac 1 {1 + 10^{(R_A - R_B)/400}} = \frac{Q_B}{Q_A + Q_B} = 1 - E_A
$$
where $Q_A = 10^{R_A/400}$ and $Q_B = 10^{R_B/400}$.

In [ ]:
def elo_prediction(elo_rank_A, elo_rank_B, base=10, scale=400):
    q_A = base**(elo_rank_A / scale)
    q_B = base**(elo_rank_B / scale)
    e_A = q_A / (q_A + q_B)
    return e_A

#### Examples

In [ ]:
p = elo_prediction(1500, 1500)
print(f"Prob. of A winning: {p:.3f}")
print(f"Prob. of B winning: {1 - p:.3f}")
print(f"Odds for A: {p / (1 - p):.3f}")

In [ ]:
p = elo_prediction(1600, 1500)
print(f"Prob. of A winning: {p:.3f}")
print(f"Prob. of B winning: {1 - p:.3f}")
print(f"Odds for A: {p / (1 - p):.3f}")

In [ ]:
p = elo_prediction(1900, 1500)
print(f"Prob. of A winning: {p:.3f}")
print(f"Prob. of B winning: {1 - p:.3f}")
print(f"Odds for A: {p / (1 - p):.3f}")

In [ ]:
p = elo_prediction(1500, 1900)
print(f"Prob. of A winning: {p:.3f}")
print(f"Prob. of B winning: {1 - p:.3f}")
print(f"Odds for A: {p / (1 - p):.3f}")

### Updated Ranking

For outcomes $S_A$ and $S_B$ (we'll assume one is a win (1) and one is a loss (0)), the updated ratings are,
\begin{gather*}
    R_A^\prime = R_A + K \times (S_A - E_A),\\
    R_B^\prime = R_B + K \times (S_B - E_B),
\end{gather*}
where $K$ is a constant chosen for the rating model that quantifies the impact of a new result on the rating.

A common value for $K$ is 32.

In [ ]:
def elo_update(rank_old, score, expected_score, k):
    return rank_old + k * (score - expected_score)

#### Examples

In [ ]:
R_A = 1500
R_B = 1500
Outcome = 1
E_A = elo_prediction(R_A, R_B)
K = 32

R_A_prime = elo_update(R_A, Outcome, E_A, K)
print(f"Old Rating: {R_A:.3f}  New Rating: {R_A_prime:.3f}")

In [ ]:
# update rating
R_A = R_A_prime
# new outcome prediction
E_A = elo_prediction(R_A, R_B)

R_A_prime = elo_update(R_A, Outcome, E_A, K)
print(f"Old Rating: {R_A:.3f}  New Rating: {R_A_prime:.3f}")

#### Order Matters
Elo Rating favors recent results

In [ ]:
# A beats B, then loses to B
R_A = 1500
R_B = 1500
E_A = E_B = .5
K = 32

# first match
Outcome = 1
R_A_prime = elo_update(R_A, Outcome, E_A, K)
R_B_prime = elo_update(R_B, 1 - Outcome, E_B, K)

# second match
E_A_prime = elo_prediction(R_A_prime, R_B_prime)
E_B_prime = 1 - E_A_prime
Outcome = 0
R_A_prime2 = elo_update(R_A_prime, Outcome, E_A_prime, K)
R_B_prime2 = elo_update(R_B_prime, 1 - Outcome, E_B_prime, K)

print(R_A_prime2, R_B_prime2)

In [ ]:
# A loses to B, then beats B
R_A = 1500
R_B = 1500
E_A = E_B = .5
K = 32

# first match
Outcome = 0
R_A_prime = elo_update(R_A, Outcome, E_A, K)
R_B_prime = elo_update(R_B, 1 - Outcome, E_B, K)

# second match
E_A_prime = elo_prediction(R_A_prime, R_B_prime)
E_B_prime = 1 - E_A_prime
Outcome = 1
R_A_prime2 = elo_update(R_A_prime, Outcome, E_A_prime, K)
R_B_prime2 = elo_update(R_B_prime, 1 - Outcome, E_B_prime, K)

print(R_A_prime2, R_B_prime2)

### Fake Data

We'll try out Elo on a small fake dataset to show how the ratings can be turned into rankings.

In [ ]:
fake_games = pd.read_csv('fake_cfb_scores.csv')
fake_games

#### EloRank

In [ ]:
from datascience_topic import elo_rank

fake_ranking = elo_rank(fake_games)
fake_ranking.sort_values(ascending=False)

### Real CFB Data

Now we'll use the actual results from the 2018 season. We drop the NESCAC schools because they only play each other and are separate.

In [ ]:
cfb_games = pd.read_csv('cfb_scores_2018.csv', parse_dates=['Date'])

nescac_schools = [
    'bates', 'amherst', 'wesleyan', 'middlebury', 'colby',
    'trinity ct', 'hamilton', 'tufts', 'bowdoin', 'williams']


nescac_mask = cfb_games['Away Team'].isin(nescac_schools) | \
    cfb_games['Home Team'].isin(nescac_schools)
date_mask = cfb_games.Date <= "2018-12-10"

cfb_games = cfb_games.loc[date_mask & ~nescac_mask].copy()

cfb_games.head(10)

### Initial Ranking: Win %

In [ ]:
from datascience_topic import win_pct_rank

rankings = pd.DataFrame()
rankings['Win %'] = win_pct_rank(cfb_games)

#### Who??

Lower division schools will pollute ranking systems.  Ideally, the ranking would handle these lower level teams but it can be hard, especially when they win every game.

In [ ]:
rankings['Elo'] = elo_rank(cfb_games)
rankings.sort_values(by='Elo', ascending=False).\
    head(25)

#### Restrict to only FBS

From now on, we'll restrict to FBS only.

In [ ]:
fbs_teams = open('d1ateams.txt').read().splitlines()
rankings.loc[fbs_teams].\
    sort_values(by='Elo', ascending=False).\
    head(25)

In [ ]:
fbs_mask = cfb_games['Away Team'].isin(fbs_teams) & \
    cfb_games['Home Team'].isin(fbs_teams)

fbs_games = cfb_games.loc[fbs_mask].copy()

fbs_rankings = pd.DataFrame()
fbs_rankings['Win %'] = win_pct_rank(cfb_games)

fbs_rankings['Elo'] = elo_rank(fbs_games)
fbs_rankings.sort_values(by='Elo', ascending=False).head(25)

## 2. Matrix/Regression Rankings

Matrix or Regression approaches utilize a matrix equation or a regression equation to produce a rating.  There are many approaches, two of which are covered here: Massey and Bradley-Terry.  Two other matrix methods are the Colley method and the Keener method.

### Massey Method

The Massey method sets up a very simple regression equation:
\begin{align*}
    \text{Home Score - Away Score} 
        & = \text{Home-Field Advantage} + \sum_{\text{All Teams}} \text{Team $i$ Rating} \times \text{Team $i$ is at Home} \\
    & \quad - \sum_{\text{All Teams}} \text{Team $i$ Rating} \times \text{Team $i$ is Away}
\end{align*}


#### Fake Data

Let's run the Massey method on our fake data

In [ ]:
fake_games = pd.read_csv('fake_cfb_scores.csv')
fake_games

In [ ]:
from datascience_topic import massey_matrix_equation, massey_rank

Massey_matrix, score_diff = massey_matrix_equation(fake_games)

Notice the structure of the Massey matrix corresponds to the above regression equation.

In [ ]:
Massey_matrix

In [ ]:
score_diff

In [ ]:
massey_fake_ranking = massey_rank(fake_games)
massey_fake_ranking.sort_values(ascending=False)

#### Real Data

We'll append the Massey ranking onto our previous table.

In [ ]:
fbs_rankings['Massey'] = massey_rank(fbs_games)
fbs_rankings.sort_values('Massey', ascending=False).head(25)

### Bradley-Terry Logistic Model

The Bradley-Terry method sets up a very _Logistic_ regression equation:
\begin{align*}
    \text{Log Odds for Home Team} 
        & = \text{Home-Field Advantage} + \sum_{\text{All Teams}} \text{Team $i$ Rating} \times \text{Team $i$ is at Home} \\
        & \quad -\sum_{\text{All Teams}} \text{Team $i$ Rating} \times \text{Team $i$ is Away}
\end{align*}

The log odds aren't observable though: we only see a team win or lose.  The objective of the Logistic Regression is to find a set of ratings where the ratings predict a high log odds for games the home team won and a low log odds for the games the home team lost.

Another way to think about it is through the logistic function:
$$
    \text{Probability Home Team Wins} = \frac{1}{1 + \exp(-\text{Log Odds for Home Team})}
$$
As the log odds goes to infinity, the probability the home team wins goes to 1.  As the log odds goes to negative infinity, the probability the home team wins goes to 0.


In [ ]:
from datascience_topic import bradleyterry_logistic_model, bradleyterry_rank

bradleyterry_matrix, outcomes = bradleyterry_logistic_model(fake_games)

The Bradley-Terry matrix is the same as the Massey matrix.

In [ ]:
bradleyterry_matrix

Now we only track wins or losses for the home team (1 is a win).

In [ ]:
outcomes

#### Bradley-Terry and Real Data

We'll append the Bradley-Terry ranking onto our previous table.

In [ ]:
fbs_rankings['BradleyTerry'] = bradleyterry_rank(fbs_games)
fbs_rankings.sort_values('BradleyTerry', ascending=False).head(25)

#### Penalized Logistic Regression

SEC West teams (and likely all Alabama opponents) are getting rated super high.  Why? Alabama won all their games and those teams won a lot of games.  If the model gives Alabama a huge rating, then the other schools can get high ratings which won't go against the fact that Alabama beat them.

Basically, this model is overfitting and we need to penalize the ratings so they aren't so large.  We used a penalized Bradley-Terry model for that.

In [ ]:
from datascience_topic import bradleyterry_penalized_rank

fbs_rankings['BradleyTerry_penalized'] = bradleyterry_penalized_rank(fbs_games)
fbs_rankings.sort_values('BradleyTerry_penalized', ascending=False).head(25)

## 3. Graph/Network Rankers

A Graph is a mathematical object that convery relations through the concept of nodes and edges.  The Python package `networkx` is incredibly useful for working with graphs.  It helps to visualize the graphs.

We're going to consider random walk/Markov chain ranking systems that utilize the graph structure to deduce the strengths of the teams.

In [ ]:
import networkx as nx
from datascience_topic import scores_to_network

graph = scores_to_network(fake_games)

In [ ]:
fake_games

In [ ]:
nx.draw_circular(graph, with_labels=True, node_color='C1')

### The Basics of the PageRank Random Walker Model

PageRank was one of the big innovation from the founders of Google, though the original ideas for the problem go back quite a ways.  PageRank is named after Larry Page from work with Sergey Brin at Stanford.  The chief idea behind PageRank is using a graph like above to quantify the importance of web pages or teams according to how often a random walker would visit that page.  Here's how that works:

**The Random Walker**

Imagine a walker who is traveling on the above graph moving from team to team.  This walker will only move randomly though and only in the proper direction of the directed edges (note how the edges are actually arrows).  If the walker is at Delaware St, there are 4 possible edges to leave on.  The walker will pick 1 at random and move to along that edge to the next team.  If the walker is at Florida, there are two edges out: Cal and Alabama.  A problem occurs if the walker reaches Alabama: there are no ways out (Alabama beat everyone).  

**Random Jump**

In order to make the random walking work, the walker on occasion will jump to a random team on the graph instead of traversing the graph.  How often will this happen?  That's a model choice but a typical value is about 15% of the time.  This random jump will allow the walker to leave Alabama eventually and keep walking.

**The Ranking**

Now imagine the walker keeps walking and walking.  If we keep track of how often the walker visits each team, then the proportion of visits gives us a ranking: the more the walker visits a team, the more important or powerful that team is in drawing in the walker, and hence the higher its ranking should be. 

In [ ]:
from datascience_topic import PageRankWalker

pr_walker = PageRankWalker(graph)
pr_walker

In [ ]:
pr_walker.walk()

#### Infinite Steps

Ideally, we'd make the walker go forever.  For small graphs, the ranking stabilizes relatively quickly.  For large graphs (like a full slate of CFB games or the internet), the random walker is rendered merely an exercise for explaining the concept.  Mathematically, we solve a eigenvalue problem using linear algebra to produce the steady-state, long-run visit probabilities.

In [ ]:
pr_walker.walk(num_steps=np.inf)

### PageRank for Real Data

Let's apply PageRank to rank CFB Games.  First we need to convert the scores to a network.

In [ ]:
cfb_graph = scores_to_network(cfb_games)

In [ ]:
from datascience_topic import draw_graph

all_divs = dict(
    FBS=open('d1ateams.txt').read().splitlines(),
    FCS=open('d1aateams.txt').read().splitlines(),
    D2=open('d2teams.txt').read().splitlines(),
    D3=open('d3teams.txt').read().splitlines(),
    NAIA=open('naia_other.txt').read().splitlines()
)

draw_graph(cfb_graph, divisions=all_divs)

In [ ]:
from datascience_topic import page_rank

rankings['PageRank'] = page_rank(cfb_graph)
rankings.sort_values('PageRank', ascending=False).head(15)

#### Division 1 Only

In [ ]:
fcs_teams = all_divs['FCS']

d1_teams = {
    'FBS': fbs_teams, 
    'FCS': fcs_teams
}

d1_graph = scores_to_network(cfb_games, divisions=d1_teams)
draw_graph(d1_graph, divisions=d1_teams)

#### FBS Only

In [ ]:
fbs_dict = {'FBS': fbs_teams}

fbs_graph = scores_to_network(cfb_games, divisions=fbs_dict)
draw_graph(fbs_graph, divisions=fbs_dict)

##### Texas and Purdue!?!?

In [ ]:
fbs_rankings['PageRank'] = page_rank(fbs_graph)
fbs_rankings.sort_values('PageRank', ascending=False).head(15)

### MonkeyRank

PageRank has a fundamental problem when it comes to ranking CFB teams.  Consider a team like Ohio State that only loses once: then when the walker arrives at Ohio State (which is often because OSU won a bunch of games), then unless the walker randomly jumps somewhere they are guaranteed to walk to Purdue next because Ohio State lost only one game, to Purdue.

A different approach developed by a friend of mine tweaks the approach of the walker (and instead thinks of it as a Monkey making picks).  The idea is that the monkey chooses _any_ opponent at random and then flips a coin to decide if he should switch from favoring the current team to favoring the opponent.  If the coin comes up heads, the monkey will favor the actual winner of the matchup.  Otherwise, he will favor the loser.  The probability of heads coming up is $p$ and is a modeling choice.  The frequency over time that the monkey favors teams determines the ranking.

The long-run steady state MonkeyRank can also be computed through solving a linear algebra problem.

In [ ]:
from datascience_topic import monkey_rank

fbs_rankings['MonkeyRank'] = monkey_rank(fbs_graph, winner_probability=.85)
fbs_rankings.sort_values('MonkeyRank', ascending=False).head(25)